# Advanced RAG Evaluation with Hybrid Search & Reranker

이 노트북은 프로젝트의 **실제 검색 로직(Hybrid Search + Reranker + Dual Query)**을 사용하여 RAG 파이프라인의 성능을 평가합니다.
기존의 단순 Vector Search 평가보다 훨씬 정확한 성능 지표를 제공합니다.

In [1]:
# !pip install ragas langchain langchain-openai qdrant-client rank_bm25 sentence-transformers

In [2]:
import os
import sys
from dotenv import load_dotenv

# 프로젝트 루트 경로 추가 (src 모듈 import 위해)
sys.path.append(os.getcwd())

# 환경 변수 로드 (.env 및 .env.local)
load_dotenv(override=True)
load_dotenv('.env.local', override=True)

from src.utils.config import ConfigDB, ConfigLLM, ConfigAPI
from src.retrievals.search_agent import execute_dual_query_search
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas import evaluate
from ragas.metrics import (
    context_precision,
    context_recall,
    faithfulness,
    answer_relevancy
)

import pandas as pd

from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_text_splitters import TokenTextSplitter
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from ragas import evaluate
from ragas.testset import TestsetGenerator
from ragas.metrics import LLMContextRecall, LLMContextPrecisionWithReference, Faithfulness, AnswerRelevancy
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from src.utils.config import ConfigDB, ConfigLLM
from src.prompts import ANALYSIS_SYSTEM_PROMPT
from src.retrievals.search_agent import execute_dual_query_search
from src.retrievals.reranker import Reranker


/Users/kim/SKN/SKN21-4th-4Team/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 393/393 [00:00<00:00, 1226.21it/s, Materializing param=roberta.encoder.layer.23.output.dense.weight]              
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_4612/4111042708.py:17: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_precision
  from ragas.metrics import (
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_4612/4111042708.py:17: DeprecationWarning: Importing context_recall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Exam

In [3]:
#################################################################
# 1. Advanced Retriever 정의 (Hybrid + Reranker)
#################################################################

def advanced_retriever(query: str):
    """
    src.retrievals.search_agent의 execute_dual_query_search를 사용하는 Retriever 함수
    - Dual Query (한글/영어)
    - Hybrid Search (Vector + Keyword + BM25)
    - Reranking (Cross-Encoder)
    """
    # 검색 실행
    results, info = execute_dual_query_search(query)

    # 관련성 점수 계산 (Inference)
    reranker = Reranker(top_k=ConfigLLM.TOP_K)
    reranked_results = reranker.invoke(query, results)

    # Dict 결과를 LangChain Document 객체로 변환
    documents = []
    for r in reranked_results:
        doc = Document(
            page_content=r['content'],
            metadata=r.get('metadata', {})
        )
        # 점수 정보도 메타데이터에 추가
        doc.metadata['score'] = r.get('score', 0)
        doc.metadata['source'] = r.get('source', '')
        documents.append(doc)
        
    return documents

# 테스트
test_docs = advanced_retriever("파인튜닝이 뭐야?")
print(f"검색된 문서 수: {len(test_docs)}")
for i, doc in enumerate(test_docs):
    print(f"[{i+1}] {doc.page_content[:100]}... (Score: {doc.metadata.get('score'):.4f})")

🔄 Reranking 진행 (후보 3개)...
Compressor 결과:
1. page_content='[강의: 1_LLM_Finetuning개요]

LLM 모델 파인튜닝(Fine tuning)  
파운데이션 모델(Foundation 모델)
**파운데이션 모델**(**foundation model**)은 **대규모 데이터**(**텍스트, 이미지, 오디오, 비디오 등**)로 사전 학습(pre-training)된 **범용 인공지능 모델**로, 다양한 하위 작업(Downstream task)를 위한 **파인튜닝**이나 **프롬프트**(**prompt**) 기반의 응답 모델에 적용할 수 있는 모델을 말한다.  
파운데이션 모델의 특징
- **광범위한 대규모 데이터로 사전 학습**
- 대형 언어 모델(LLM)은 방대한 양의 텍스트 데이터로 사전 학습(pre-training)되며, 그 결과 **다양한 주제와 여러 언어에 대한 일반적인 지식과 언어 패턴등 언어에 대한 범용적 지식**을 포함하고 있다.
- 이미지 기반 파운데이션 모델(image-based foundation model) 또한 대규모 이미지·영상 데이터에서 사물, 장면, 물체 구조의 시각적 특징을 학습하여, **개별 사물의 특성(예: 형태, 질감, 맥락)과 물체들이 공유하는 일반적 물리 특성(예: 경계, 입체감, 재질, 공간 관계)을 표현 공간(representation space)에 반영**한다.
- **대규모 모델**
- 파운데이션 모델은 대부분 **Transformer** 구조를 기반(특히 자연어 모델)으로 하며, **수십억에서 수조개의 파라미터** 규모를 가지고 있다.
- **모델의 크기가 클 수록 성능과 표현력이 향상된다.**
- **범용성**
- **특정 작업에 국한 되지 않고**, 다양한 자연어 처리 작업에 적용될 수 있는 기초 역량을 제공한다.
- 일반적인 NLP task인 문장 생성, 문서요약, 번역, 질의응답에서 좋은 성능을 보인다.
- **프롬프트 기반 실행**
- 추가적인 학습(파인튜닝)

In [4]:
llm_context = """당신은 Python 프로그래밍 교육 과정의 **AI 조교(Teaching Assistant)**이자 데이터셋 생성 전문가입니다.
제공된 [강의 자료(Lecture)]와 [Python 공식 문서(RST)]를 기반으로, 학습자 평가를 위한 고품질의 JSON 포맷 질의응답(QA) 데이터셋을 생성하십시오.

생성 시 다음의 **지침(Instruction)**과 답변 스타일(Tone & Style)을 철저히 준수해야 합니다.

---

"""

llm_context += ANALYSIS_SYSTEM_PROMPT

llm_context += """

---

## QA 생성 제약 (Constraints)
- **언어**: 가능하면 한국어로 질문하고 답변하되, 코드 예제는 영어로 작성하십시오.
- **문법**: 문장의 끝은 마침표(.) 등 **구두점을 반드시 표기**하여 완결된 문장으로 작성하십시오.
- **JSON 포맷 엄수**:
  - 출력은 반드시 파싱 가능한 **JSON 배열** 형태여야 합니다 (`[{"user_input": "...", "reference": "..."}, ...]`).
  - **이스케이프 처리**: 본문 내의 큰따옴표("), 백슬래시(\) 등 특수 문자는 반드시 역슬래시(\")를 사용하여 이스케이프 처리하십시오.
  - 소스 텍스트에 JSON 문법을 해치는 요소가 있더라도, 최종 출력은 유효한 JSON이어야 합니다.

---

## QA 생성 출력 예시
[
  {
    "user_input": "파인튜닝의 장점은 무엇인가?",
    "reference": "파인튜닝은 사전 학습된 모델(Foundation 모델)을 특정 태스크나 도메인 데이터로 추가 학습 최적화 하는 과정으로, 특정 작업/도메인에 최적화, 사용자 맞춤형 톤앤매너 적용, 안전성과 윤리 강화를 위해 필요하다."
  }
]

위 가이드를 바탕으로 주어진 텍스트에서 학습 가치가 높은 QA 세트를 생성하십시오.
"""

<>:21: SyntaxWarning: invalid escape sequence '\)'
<>:21: SyntaxWarning: invalid escape sequence '\)'
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_4612/3359774379.py:21: SyntaxWarning: invalid escape sequence '\)'
  - **이스케이프 처리**: 본문 내의 큰따옴표("), 백슬래시(\) 등 특수 문자는 반드시 역슬래시(\")를 사용하여 이스케이프 처리하십시오.


In [5]:
client = QdrantClient(host='localhost', port=ConfigDB.PORT)
info = client.get_collection(ConfigDB.COLLECTION_NAME)

results, next_id = client.scroll(
    collection_name=ConfigDB.COLLECTION_NAME,
    limit=info.points_count,
)

# sampling
import random
sample_dataset = random.sample(results, 50) # 리스트에서 랜덤하게 K개를 추출


# 문서 내용만 추출
docs = [point.payload['page_content'] for point in sample_dataset if point.payload.get('metadata', {}).get('source', '') == 'lecture']
docs

['[강의: 09_CNN_개요]\n\n- Filter/Kernel',
 '[강의: 01_딥러닝 개요]\n\n딥러닝의 특징  \n- 기존 머신러닝과 딥러닝은 모두 데이터를 학습시켜 모델을 구축한다는 공통점을 가진다.\n- 효과적인 학습을 위해서는 데이터로부터 목표에 적합한 특성을 추출하여 학습 데이터셋을 잘 구성해야 한다.\n- 원본 데이터(raw data)에는 패턴 인식에 불필요하거나 방해가 되는 노이즈(noise)가 포함되어 있다.\n- 따라서 데이터 전처리를 통해 노이즈를 제거하고 유의미한 특성을 추출해야 한다. 이러한 과정을 **데이터 전처리(data preprocessing)** 또는 **특성 추출(feature extraction)** 이라고 한다.\n- 기존 머신러닝에서는 특성 추출(feature extraction)을 사람이 직접 수행하고, 이후에 생성된 특성 벡터(feature vector)를 기반으로 모델이 학습된다.\n- 딥러닝은 모델이 학습 과정에서 특성 추출과 모델 학습을 동시에 수행한다.\n- 이러한 구조 덕분에 딥러닝은 이미지, 음성, 텍스트 등과 같은 특성 추출을 잘 하기가 어려운운 비정형 데이터에서 우수한 성능을 보인다. 반면, 특성 추출이 상대적으로 쉬운 정형 데이터에서는 기존 머신러닝이 더 효율적일 수 있다.',
 "[강의: 07_모델저장_문제 유형별 모델 생성]\n\n위스콘신 유방암 데이터셋 - **이진분류(Binary Classification) 문제**  \n- **이진 분류 문제 처리 모델의 두가지 방법**\n1. positive(1)일 확률을 출력하도록 구현\n- output layer: units=1, activation='sigmoid'\n- loss: binary_crossentropy\n2. negative(0)일 확률과 positive(1)일 확률을 출력하도록 구현 => 다중분류 처리 방식으로 해결\n- output layer: units=2, activation='softmax', y(정답)은 one hot 

In [6]:
#################################################################
# 2. Testset 준비
# (기존에 생성해둔 평가 데이터셋을 사용하거나 새로 생성)
#################################################################
import pandas as pd
from datasets import Dataset

generator_llm = LangchainLLMWrapper(ChatOpenAI(model=ConfigLLM.OPENAI_MODEL))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=ConfigDB.EMBEDDING_MODEL))

generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    llm_context=llm_context 
)

# testset
testset = generator.generate_with_chunks(
    docs, testset_size=20  # context 내용, 테스트데이터셋 몇개를 만들지.
)
testset


/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_4612/2408905276.py:8: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  generator_llm = LangchainLLMWrapper(ChatOpenAI(model=ConfigLLM.OPENAI_MODEL))
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_4612/2408905276.py:9: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=ConfigDB.EMBEDDING_MODEL))
Applying CustomNodeFilter:   0%|          | 0/7 [00:00<?, ?it/s]Node b78d7f90-8fa5-48b2-bf42

Testset(samples=[TestsetSample(eval_sample=SingleTurnSample(user_input='데이터 전처리의 중요성에 대해 설명해 주세요.', retrieved_contexts=None, reference_contexts=['[강의: 01_딥러닝 개요]\n\n딥러닝의 특징  \n- 기존 머신러닝과 딥러닝은 모두 데이터를 학습시켜 모델을 구축한다는 공통점을 가진다.\n- 효과적인 학습을 위해서는 데이터로부터 목표에 적합한 특성을 추출하여 학습 데이터셋을 잘 구성해야 한다.\n- 원본 데이터(raw data)에는 패턴 인식에 불필요하거나 방해가 되는 노이즈(noise)가 포함되어 있다.\n- 따라서 데이터 전처리를 통해 노이즈를 제거하고 유의미한 특성을 추출해야 한다. 이러한 과정을 **데이터 전처리(data preprocessing)** 또는 **특성 추출(feature extraction)** 이라고 한다.\n- 기존 머신러닝에서는 특성 추출(feature extraction)을 사람이 직접 수행하고, 이후에 생성된 특성 벡터(feature vector)를 기반으로 모델이 학습된다.\n- 딥러닝은 모델이 학습 과정에서 특성 추출과 모델 학습을 동시에 수행한다.\n- 이러한 구조 덕분에 딥러닝은 이미지, 음성, 텍스트 등과 같은 특성 추출을 잘 하기가 어려운운 비정형 데이터에서 우수한 성능을 보인다. 반면, 특성 추출이 상대적으로 쉬운 정형 데이터에서는 기존 머신러닝이 더 효율적일 수 있다.'], retrieved_context_ids=None, reference_context_ids=None, response=None, multi_responses=None, reference='데이터 전처리는 원본 데이터(raw data)에서 노이즈(noise)를 제거하고 유의미한 특성을 추출하는 과정입니다. 이는 효과적인 학습을 위해 목표에 적합한 특성을 추출하여 학습 데이터셋을 잘 구성하는 데 필수적입니다. 딥러닝에서는 모델이 학습 과정

In [7]:
eval_df = testset.to_pandas()

# Reference 데이터 재생성 (LangGraph 적용)
from src.agent.workflow import build_reference_generation_graph
from langchain_core.messages import HumanMessage

# Graph 빌드
ref_graph = build_reference_generation_graph()
app = ref_graph.compile()

print("Reference 데이터 재생성 중...")
new_references = []
for idx, row in eval_df.iterrows():
    if isinstance(row['reference_contexts'], list):
        context_str = "\n\n".join(row['reference_contexts'])
    else:
        context_str = str(row['reference_contexts'])
    
    query = row['user_input']
    
    # Generate
    try:
        # LangGraph 실행
        result = app.invoke({"context": context_str, "query": query})
        # 결과 추출
        if result.get("analyst_results"):
             new_ref = result["analyst_results"][-1].content
        else:
             new_ref = ""
    except Exception as e:
        print(f"Error generating reference for query {query}: {e}")
        new_ref = row.get('reference', '') # Fallback to original
        
    new_references.append(new_ref)

eval_df['reference'] = new_references

# 다음 셀 호환성을 위한 데이터 준비
eval_data = eval_df.rename(columns={'user_input': 'question'}).to_dict('list')

eval_df.head()

Reference 데이터 재생성 중...


,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name
0,데이터 전처리의 중요성에 대해 설명해 주세요.,[[강의: 01_딥러닝 개요]\n\n딥러닝의 특징 \n- 기존 머신러닝과 딥러닝은...,**핵심 답변**: 데이터 전처리는 원본 데이터에서 노이즈를 제거하고 유의미한 특성...,Deep Learning Researcher,MISSPELLED,LONG,single_hop_specific_query_synthesizer
1,특성 추출이란 무엇인가요?,[[강의: 01_딥러닝 개요]\n\n딥러닝의 특징 \n- 기존 머신러닝과 딥러닝은...,**핵심 답변**: 특성 추출(feature extraction)은 원본 데이터에서...,Deep Learning Researcher,WEB_SEARCH_LIKE,SHORT,single_hop_specific_query_synthesizer
2,머신러닝과 딥러닝의 공통점은 무엇인가요?,[[강의: 01_딥러닝 개요]\n\n딥러닝의 특징 \n- 기존 머신러닝과 딥러닝은...,**핵심 답변**: 머신러닝과 딥러닝의 공통점은 모두 데이터를 학습시켜 모델을 구축...,Deep Learning Researcher,PERFECT_GRAMMAR,MEDIUM,single_hop_specific_query_synthesizer
3,위스콘신 대학교에서 제공한 유방암 데이터셋은 어떤 이진 분류 문제를 다루고 있나요?,[[강의: 07_모델저장_문제 유형별 모델 생성]\n\n위스콘신 유방암 데이터셋 -...,위스콘신 대학교에서 제공한 유방암 데이터셋은 종양의 악성 여부를 이진 분류하는 문제...,Data Scientist,POOR_GRAMMAR,LONG,single_hop_specific_query_synthesizer
4,위스콘신 유방암 데이터셋은 뭐에 쓰는거야?,[[강의: 07_모델저장_문제 유형별 모델 생성]\n\n위스콘신 유방암 데이터셋 -...,위스콘신 유방암 데이터셋은 종양의 악성 여부를 분류하기 위해 사용됩니다. 이 데이터...,Data Scientist,POOR_GRAMMAR,SHORT,single_hop_specific_query_synthesizer


In [8]:
#################################################################
# 3. RAG 파이프라인 구성 및 실행
#################################################################
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from src.prompts import ANALYSIS_SYSTEM_PROMPT
from langchain_core.runnables import RunnableLambda

# LLM 설정
llm = ChatOpenAI(model=ConfigLLM.OPENAI_MODEL, temperature=0)

# 프롬프트
prompt = ChatPromptTemplate.from_template(ANALYSIS_SYSTEM_PROMPT)

# 문서 포맷팅
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain 구성
rag_chain = (
    {
        "context": RunnableLambda(advanced_retriever) | RunnableLambda(format_docs), 
        "query": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

# 답변 생성 및 Context 수집
answers = []
contexts = []

print("평가 데이터에 대한 응답 생성 중...")
for q in eval_data['question']:
    # 1. 문서 검색
    retrieved_docs = advanced_retriever(q)
    context_text = [doc.page_content for doc in retrieved_docs]
    contexts.append(context_text)
    
    # 2. 답변 생성
    # Chain을 직접 호출하면 context를 재검색하므로(retriever가 chain 안에 있음),
    # 효율성을 위해 미리 검색한 docs를 주입하거나, 그냥 chain을 invoke합니다.
    # 여기서는 정확한 평가를 위해 chain invoke 결과를 사용합니다.
    response = rag_chain.invoke(q)
    answers.append(response)
    # print(f"Q: {q}\nA: {response[:50]}...\n")

# 데이터셋에 결과 추가
eval_data['answer'] = answers
eval_data['contexts'] = contexts

final_dataset = Dataset.from_dict(eval_data)

평가 데이터에 대한 응답 생성 중...
🔄 Reranking 진행 (후보 3개)...
Compressor 결과:
1. page_content='[강의: 04_데이터_전처리]

Data 전처리(Data Preprocessing)란  
- 데이터 분석이나 머신러닝 모델에 적합한 형태로 데이터셋을 변환 또는 조정하는 과정을 말한다.
- 데이터 분석, 머신러닝 모델링 전에 수행하는 작업이다.
- Garbage in, Garbage out.
- 좋은 dataset으로 학습 해야 좋은 예측 결과를 만드는 모델을 학습할 수 있다.
- 좋은 train dataset을 만드는 것은 모델의 성능에 가장 큰 영향을 준다.
- Data 전처리에는 다음과 같은 작업이 있다.
- **Data Cleaning (데이터 정제)**
- 데이터셋에 있는 오류값, 불필요한 값, 결측치, 중복값 등을 제거하는 작업
- **컬럼 선택 및 파생변수 생성**
- 컬럼들 중 분석에 필요한 컬럼들만 선택하거나 기존 컬럼들을 계산한 결과값을 가지는 파생변수를 생성한다.
- **Feature의 데이터 타입 별 변환**
- 문자열을 날짜 타입으로 변환, 범주형을 수치형으로 변환등과 같이 원래 데이터의 형식에 맞게 변환하는 작업.
- **수치형 데이터 Feature Scaling**
- 수치형 컬럼들의 scale(척도) 를 맞춰 주는 작업.
- **범주형 데이터 인코딩**
- 문자열 형태로 되어있는 범주형 데이터를 숫자 형태로 변경하는 작업.' metadata={'source': '', 'source_file': '04_데이터_전처리.ipynb', 'lecture_title': '04_데이터_전처리', 'cell_type': 'markdown', 'cell_index': 0, 'code_snippet': '', 'chunk_index': 0, 'original_score': 0.5783624}
   점수: 0.0
   원본 점수: 0.5783624
-----------------------------------

In [9]:
#################################################################
# 4. Ragas 평가 실행
#################################################################

result = evaluate(
    final_dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
    llm=llm,
    embeddings=OpenAIEmbeddings(model=ConfigDB.EMBEDDING_MODEL)
)

print("\n========== 평가 결과 ==========")
print(result)
df_result = result.to_pandas()
df_result

Evaluating:  38%|███▊      | 23/60 [00:39<00:39,  1.08s/it]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating: 100%|██████████| 60/60 [01:42<00:00,  1.70s/it]



========== 평가 결과 ==========
{'context_precision': 0.9758, 'context_recall': 0.7944, 'faithfulness': 0.8450, 'answer_relevancy': 0.5700}


,user_input,retrieved_contexts,reference_contexts,response,reference,persona_name,query_style,query_length,context_precision,context_recall,faithfulness,answer_relevancy
0,데이터 전처리의 중요성에 대해 설명해 주세요.,[[강의: 01_머신러닝개요]\n\n3. Data Preparation\n- 데이터...,[[강의: 01_딥러닝 개요]\n\n딥러닝의 특징 \n- 기존 머신러닝과 딥러닝은...,데이터 전처리는 머신러닝 모델의 성능에 큰 영향을 미치기 때문에 매우 중요합니다. ...,**핵심 답변**: 데이터 전처리는 원본 데이터에서 노이즈를 제거하고 유의미한 특성...,Deep Learning Researcher,MISSPELLED,LONG,1.000000,0.750000,1.000,0.604906
1,특성 추출이란 무엇인가요?,[[강의: 01_딥러닝 개요]\n\n딥러닝의 특징 \n- 기존 머신러닝과 딥러닝은...,[[강의: 01_딥러닝 개요]\n\n딥러닝의 특징 \n- 기존 머신러닝과 딥러닝은...,**핵심 답변**: 특성 추출(feature extraction)은 데이터에서 유의...,**핵심 답변**: 특성 추출(feature extraction)은 원본 데이터에서...,Deep Learning Researcher,WEB_SEARCH_LIKE,SHORT,1.000000,1.000000,0.800,0.434434
2,머신러닝과 딥러닝의 공통점은 무엇인가요?,[[강의: 01_딥러닝 개요]\n\n머신러닝과 딥러닝 \n머신러닝(Machine ...,[[강의: 01_딥러닝 개요]\n\n딥러닝의 특징 \n- 기존 머신러닝과 딥러닝은...,**핵심 답변**: 머신러닝과 딥러닝의 공통점은 둘 다 데이터를 학습하여 알고리즘을...,**핵심 답변**: 머신러닝과 딥러닝의 공통점은 모두 데이터를 학습시켜 모델을 구축...,Deep Learning Researcher,PERFECT_GRAMMAR,MEDIUM,1.000000,0.666667,0.500,0.663401
3,위스콘신 대학교에서 제공한 유방암 데이터셋은 어떤 이진 분류 문제를 다루고 있나요?,[[강의: 07_모델저장_문제 유형별 모델 생성]\n\n위스콘신 유방암 데이터셋 -...,[[강의: 07_모델저장_문제 유형별 모델 생성]\n\n위스콘신 유방암 데이터셋 -...,위스콘신 대학교에서 제공한 유방암 데이터셋은 종양의 악성 여부를 분류하는 이진 분류...,위스콘신 대학교에서 제공한 유방암 데이터셋은 종양의 악성 여부를 이진 분류하는 문제...,Data Scientist,POOR_GRAMMAR,LONG,1.000000,0.800000,1.000,0.587451
4,위스콘신 유방암 데이터셋은 뭐에 쓰는거야?,[[강의: 05_평가지표]\n\nTODO: breast_cancer data 모델링...,[[강의: 07_모델저장_문제 유형별 모델 생성]\n\n위스콘신 유방암 데이터셋 -...,"위스콘신 유방암 데이터셋은 유방암 진단을 위한 데이터로, 주로 암의 악성 여부를 예...",위스콘신 유방암 데이터셋은 종양의 악성 여부를 분류하기 위해 사용됩니다. 이 데이터...,Data Scientist,POOR_GRAMMAR,SHORT,1.000000,0.200000,0.375,0.570012
5,위스콘신 유방암 데이터셋의 주요 특징은 무엇인가요?,[[강의: 05_평가지표]\n\nTODO: breast_cancer data 모델링...,[[강의: 07_모델저장_문제 유형별 모델 생성]\n\n위스콘신 유방암 데이터셋 -...,"핵심 답변: 위스콘신 유방암 데이터셋은 유방암 진단 결과를 포함하고 있으며, 주요 ...",**핵심 답변**: 위스콘신 유방암 데이터셋의 주요 특징은 종양에 대한 다양한 측정...,Data Scientist,PERFECT_GRAMMAR,SHORT,1.000000,1.000000,0.500,0.515874
6,강의에서 신경망 구조의 학습 프로세스에 대해 설명해 줄 수 있나요?,"[[강의: 05_신경망 구조]\n\nTrain(학습) 프로세스, [강의: 01_딥러...",[[강의: 05_신경망 구조]\n\nTrain(학습) 프로세스],신경망 구조의 학습 프로세스는 크게 두 가지 단계로 나눌 수 있습니다: **순전파(...,"신경망 구조의 학습 프로세스는 주로 데이터의 입력, 가중치 조정, 손실 함수 계산,...",Data Scientist,MISSPELLED,LONG,0.804167,0.600000,1.000,0.692942
7,신경망 구조의 학습 프로세스는 무엇인가요?,"[[강의: 05_신경망 구조]\n\nTrain(학습) 프로세스, [강의: 01_딥러...",[[강의: 05_신경망 구조]\n\nTrain(학습) 프로세스],**핵심 답변**: 신경망 구조의 학습 프로세스는 순전파(forward propag...,**핵심 답변**: 신경망 구조의 학습 프로세스는 주어진 데이터로부터 모델의 가중치...,Deep Learning Researcher,PERFECT_GRAMMAR,MEDIUM,1.000000,0.800000,1.000,0.472914
8,Train(학습) 프로세스는 뭐에요?,"[[강의: 05_신경망 구조]\n\nTrain(학습) 프로세스, [강의: 04_첫번...",[[강의: 05_신경망 구조]\n\nTrain(학습) 프로세스],**핵심 답변**: Train(학습) 프로세스는 모델이 주어진 데이터로부터 패턴을 ...,**핵심 답변**: Train(학습) 프로세스는 신경망 모델이 주어진 데이터로부터 ...,Deep Learning Researcher,POOR_GRAMMAR,MEDIUM,1.000000,0.600000,0.500,0.600810
9,재현율이 중요한 경우는 어떤 상황인가요?,[[강의: 05_평가지표]\n\n재현율과 정밀도의 관계 \n**분류의 경우 Pre...,[[강의: 05_평가지표]\n\n재현율이 더 중요한 경우\n- 실제 Positive...,재현율이 중요한 경우는 실제 Positive 데이터를 Negative로 잘못 판단했...,**핵심 답변**: 재현율이 중요한 경우는 실제 Positive 데이터를 Negat...,Data Scientist,WEB_SEARCH_LIKE,MEDIUM,1.000000,0.500000,1.000,0.999999


In [ ]:
# {'context_precision': 0.8333, 'context_recall': 0.7044, 'faithfulness': 0.8240, 'answer_relevancy': 0.5833}
# {'context_precision': 0.9758, 'context_recall': 0.7944, 'faithfulness': 0.8450, 'answer_relevancy': 0.5700}